### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [1]:
import numpy as np
import pandas as pd

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [30]:
movies.columns

Index(['movie_id', 'movie', 'genre', 'date', '1800's', '1900's', '2000's',
       'History', 'News', 'Horror', 'Musical', 'Film-Noir', 'Mystery',
       'Adventure', 'Sport', 'War', 'Music', 'Reality-TV', 'Adult', 'Crime',
       'Family', 'Drama', 'Talk-Show', 'Biography', 'Sci-Fi', 'Fantasy',
       'Romance', 'Game-Show', 'Action', 'Documentary', 'Animation', 'Comedy',
       'Short', 'Western', 'Thriller'],
      dtype='object')

In [27]:
reviews.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'date'], dtype='object')

In [33]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,114508,8,1381006850,2013-10-05 17:00:50
1,2,102926,9,1590148016,2020-05-22 07:46:56
2,2,208092,5,1586466072,2020-04-09 17:01:12
3,2,358273,9,1579057827,2020-01-14 22:10:27
4,2,10039344,5,1578603053,2020-01-09 15:50:53


In [40]:
print(np.array(reviews.index[:5]))

[0 1 2 3 4]


In [28]:
reviews.iloc[:200].shape, reviews.iloc[200:].shape,reviews.shape

((200, 5), (876165, 5), (876365, 5))

1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [16]:
from sklearn.model_selection import train_test_split

def create_train_test(reviews, order_by, training_size, testing_size):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of columns in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    sorted_revs = reviews.sort_values(order_by)
    sorted_revs = sorted_revs.iloc[:training_size+testing_size]
    training_df, validation_df = sorted_revs.iloc[:training_size],sorted_revs.iloc[training_size:]
    return training_df, validation_df

In [17]:
# Nothing to change in this or the next cell
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [18]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-03-15', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-03-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

AssertionError: The last date in the training dataset doesn't look like what we expected.

In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 250 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.  No need to change the code below unless you would like to make changes to reduce the time needed to obtain results.

In [19]:
def FunkSVD(ratings_mat, latent_features=12, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

In [23]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.array(train_data_df)

# Fit FunkSVD with the specified hyper parameters to the training data
# user_mat, movie_mat = FunkSVD(train_data, latent_features=15, learning_rate=0.005, iters=250)

import time

start_time = time.time()
user_mat, movie_mat = FunkSVD(train_data_np, latent_features=5, learning_rate=0.01, iters=50)

print("total run time = {}".format(time.time()-start_time))


Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 24.714637
2 		 13.074159
3 		 8.264757
4 		 5.616925
5 		 3.937402
6 		 2.808202
7 		 2.033367
8 		 1.499606
9 		 1.131406
10 		 0.875874
11 		 0.696066
12 		 0.566876
13 		 0.471723
14 		 0.399819
15 		 0.344134
16 		 0.300022
17 		 0.264356
18 		 0.234990
19 		 0.210420
20 		 0.189579
21 		 0.171691
22 		 0.156185
23 		 0.142633
24 		 0.130707
25 		 0.120149
26 		 0.110755
27 		 0.102361
28 		 0.094831
29 		 0.088052
30 		 0.081928
31 		 0.076381
32 		 0.071341
33 		 0.066750
34 		 0.062558
35 		 0.058722
36 		 0.055205
37 		 0.051974
38 		 0.049000
39 		 0.046260
40 		 0.043729
41 		 0.041390
42 		 0.039225
43 		 0.037217
44 		 0.035354
45 		 0.033621
46 		 0.032009
47 		 0.030506
48 		 0.029104
49 		 0.027793
50 		 0.026566
total run time = 183.5876064300537


In [43]:
user_mat.shape, movie_mat.shape

train_data_df

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [69]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''
    # Create series of users and movies in the right order
    
    user_ser = pd.Series(train_data_df.index)
    movie_ser = pd.Series(train_data_df.columns)    

    # User row and Movie Column
    if (np.where(user_ser==user_id)[0]).size > 0 and (np.where(movie_ser==movie_id)[0]).size > 0:
        user_row = user_matrix[np.where(user_ser==user_id)[0][0],:]
        movie_col = movie_matrix[:,np.where(movie_ser==movie_id)[0][0]]

        # Take dot product of that row and column in U and V to make prediction
        pred = np.dot(user_row,movie_col)
    
    else:
        print("either user_id or movie_id not present in data")
        
        pred = None
    
    return pred

In [90]:
# Test your function with the first user-movie in the user-movie matrix (notice this is a nan)
pred_val = predict_rating(user_mat, movie_mat, 27, 2844)
pred_val

5.705779644153555

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [88]:
movies[movies['movie_id']==2844]['movie'].iloc[0]

"Fantômas - À l'ombre de la guillotine (1913)"

In [101]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    prediction - the predicted rating for user_id-movie_id
    
    OUTPUT:
    None - prints a statement about the user, movie, and prediction made
    
    '''
    
    mv_name = movies[movies['movie_id']==2844]['movie'].iloc[0]
    
    print("user_id {} is expected to give ranking {} to movie '{}'".format(user_id, prediction, mv_name))
    

In [102]:
# Test your function the the results of the previous function
print_prediction_summary(8, 2844, pred_val)

user_id 8 is expected to give ranking 5.705779644153555 to movie 'Fantômas - À l'ombre de la guillotine (1913)'


Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [108]:
val_df.head()

,user_id,movie_id,rating,timestamp,date
555303,43567,289879,9,1363308742,2013-03-14 20:52:22
555319,43567,1563738,9,1363308780,2013-03-14 20:53:00
555317,43567,1458175,4,1363308799,2013-03-14 20:53:19
757959,59557,103639,8,1363309112,2013-03-14 20:58:32
555416,43573,1560985,4,1363309202,2013-03-14 21:00:02


In [109]:
train_df.tail()

,user_id,movie_id,rating,timestamp,date
475448,37514,1790885,8,1363308196,2013-03-14 20:43:16
490982,38767,1194173,6,1363308215,2013-03-14 20:43:35
555316,43567,1411238,7,1363308646,2013-03-14 20:50:46
555309,43567,1028532,5,1363308697,2013-03-14 20:51:37
555322,43567,1598822,8,1363308721,2013-03-14 20:52:01


In [116]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made for each row of val_df from row 0 to num_preds
    '''
    # first num_preds rows
    val_df_sliced = val_df[:num_preds]
#     print(val_df_sliced)
    
    for i in range(0,len(val_df_sliced)):
        row = val_df_sliced.iloc[i]
        # get user_id and movie id
        
#         print(row)
        uid, mvid = row['user_id'],row['movie_id']
        pred_val = predict_rating(user_mat, movie_mat, uid, mvid)
        
        print("user {} ranked movie {} at {} and was expected to rank at {}".format(\
                        uid, mvid, row['rating'], pred_val))
        
# Perform the predicted vs. actual for the first 6 rows.  How does it look?
validation_comparison(val_df, 6)        

user 43567 ranked movie 289879 at 9 and was expected to rank at 9.056967730024962
user 43567 ranked movie 1563738 at 9 and was expected to rank at 7.831772847084651
user 43567 ranked movie 1458175 at 4 and was expected to rank at 8.161758236375237
user 59557 ranked movie 103639 at 8 and was expected to rank at 7.595497214071126
user 43573 ranked movie 1560985 at 4 and was expected to rank at 3.438113740245925
either user_id or movie_id not present in data
user 4902 ranked movie 287978 at 9 and was expected to rank at None


In [119]:
# Perform the predicted vs. actual for the first 7 rows.  What happened?
validation_comparison(val_df, 7)        

user 43567 ranked movie 289879 at 9 and was expected to rank at 9.056967730024962
user 43567 ranked movie 1563738 at 9 and was expected to rank at 7.831772847084651
user 43567 ranked movie 1458175 at 4 and was expected to rank at 8.161758236375237
user 59557 ranked movie 103639 at 8 and was expected to rank at 7.595497214071126
user 43573 ranked movie 1560985 at 4 and was expected to rank at 3.438113740245925
either user_id or movie_id not present in data
user 4902 ranked movie 287978 at 9 and was expected to rank at None
user 1246 ranked movie 1028532 at 8 and was expected to rank at 2.901403437329399


** A statement about why you think what happened happened.**